source: https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive/05_artandscience/d_customestimator_linear.ipynb

A few changes have been made to the original notebook file.

## Result

Using the custom DNN Regressor estimator below the result was in one run (house price scaled with 1/100000):

global step 10000: global_step = 10000, loss = 0.41009486, rmse = 0.64038646


# Custom Estimator

**Learning Objectives:**
  * Use a custom estimator of the `Estimator` class in TensorFlow to predict median housing price

The data is based on 1990 census data from California. This data is at the city block level, so these features reflect the total number of rooms in that block, or the total number of people who live on that block, respectively.
<p>
Let's use a set of features to predict house value.

## Set Up
In this first cell, we'll load the necessary libraries.

In [12]:
import math
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf

import os

from google.datalab.ml import TensorBoard

print(tf.__version__)

tf.logging.set_verbosity(tf.logging.INFO)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

1.12.0


Next, we'll load our data set.

In [25]:
#df = pd.read_csv("https://storage.googleapis.com/ml_universities/california_housing_train.csv", sep = ",")

df = pd.read_csv("data/california_housing_train.csv", sep=",")

## Examine the data

It's a good idea to get to know your data a little bit before you work with it.

We'll print out a quick summary of a few useful statistics on each column.

This will include things like mean, standard deviation, max, min, and various quantiles.

In [26]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.3,34.2,15.0,5612.0,1283.0,1015.0,472.0,1.5,66900.0
1,-114.5,34.4,19.0,7650.0,1901.0,1129.0,463.0,1.8,80100.0
2,-114.6,33.7,17.0,720.0,174.0,333.0,117.0,1.7,85700.0
3,-114.6,33.6,14.0,1501.0,337.0,515.0,226.0,3.2,73400.0
4,-114.6,33.6,20.0,1454.0,326.0,624.0,262.0,1.9,65500.0


In [27]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207300.9
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,115983.8
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,14999.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119400.0
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180400.0
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265000.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500001.0


This data is at the city block level, so these features reflect the total number of rooms in that block, or the total number of people who live on that block, respectively.  Let's create a different, more appropriate feature.  Because we are predicing the price of a single house, we should try to make all our features correspond to a single house as well

In [28]:
df['num_rooms'] = df['total_rooms'] / df['households']
df['num_bedrooms'] = df['total_bedrooms'] / df['households']
df['persons_per_house'] = df['population'] / df['households']

df['median_house_value'] = df['median_house_value'] / 100000

df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,num_rooms,num_bedrooms,persons_per_house
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,2.1,5.4,1.1,3.0
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,1.2,2.5,0.5,4.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,0.1,0.8,0.3,0.7
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,1.2,4.4,1.0,2.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,1.8,5.2,1.0,2.8
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,2.6,6.1,1.1,3.3
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,5.0,141.9,34.1,502.5


In [29]:
df.drop(['total_rooms', 'total_bedrooms', 'population', 'households'], axis = 1, inplace = True)
df.describe()

,longitude,latitude,housing_median_age,median_income,median_house_value,num_rooms,num_bedrooms,persons_per_house
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,3.9,2.1,5.4,1.1,3.0
std,2.0,2.1,12.6,1.9,1.2,2.5,0.5,4.0
min,-124.3,32.5,1.0,0.5,0.1,0.8,0.3,0.7
25%,-121.8,33.9,18.0,2.6,1.2,4.4,1.0,2.4
50%,-118.5,34.2,29.0,3.5,1.8,5.2,1.0,2.8
75%,-118.0,37.7,37.0,4.8,2.6,6.1,1.1,3.3
max,-114.3,42.0,52.0,15.0,5.0,141.9,34.1,502.5


## Build a custom estimator linear regressor

In this exercise, we'll be trying to predict `median_house_value`. It will be our label. We'll use the remaining columns as our input features.

To train our model, we'll use the Estimator API and create a custom estimator for linear regression.

Note that we don't actually need a custom estimator for linear regression since there is a canned estimator for it, however we're keeping it simple so you can practice creating a custom estimator function.

In [30]:
# Define feature columns
feature_columns = {
  colname : tf.feature_column.numeric_column(colname) \
    for colname in ['housing_median_age','median_income','num_rooms','num_bedrooms','persons_per_house']
}
# Bucketize lat, lon so it's not so high-res; California is mostly N-S, so more lats than lons
feature_columns['longitude'] = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('longitude'), np.linspace(-124.3, -114.3, 10).tolist())
feature_columns['latitude'] = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('latitude'), np.linspace(32.5, 42, 10).tolist())

In [31]:
# Split into train and eval and create input functions
msk = np.random.rand(len(df)) < 0.8
traindf = df[msk]
evaldf = df[~msk]

BATCH_SIZE=128

# Returns input function that would feed Pandas DataFrame into the model.

train_input_fn = tf.estimator.inputs.pandas_input_fn(x = traindf[list(feature_columns.keys())],
                                                    y = traindf["median_house_value"],
                                                    num_epochs = None,
                                                    batch_size = BATCH_SIZE,
                                                    shuffle = True)

eval_input_fn = tf.estimator.inputs.pandas_input_fn(x = evaldf[list(feature_columns.keys())],
                                                    y = evaldf["median_house_value"],  # note the scaling
                                                    num_epochs = 1, 
                                                    batch_size = len(evaldf), 
                                                    shuffle=False)

In [42]:
# tf.feature_column.input_layer(): A Tensor which represents input layer of a model. 
#   Its shape is (batch_size, first_layer_dimension) and its dtype is float32. 
#   first_layer_dimension is determined based on given feature_columns.

# tf.estimator.EstimatorSpec() fully defines the model to be run by an Estimator.


# Create the custom estimator
def custom_estimator(features, labels, mode, params):
    
  # 0. Extract data from feature columns
  input_layer = tf.feature_column.input_layer(features, params['feature_columns'])
  
  # 1. Define Model Architecture
  if mode == tf.estimator.ModeKeys.TRAIN:  

      predictions = tf.layers.dense(input_layer, 128, activation=tf.nn.tanh)
      predictions = tf.layers.dropout(predictions, rate=0.15)

      predictions = tf.layers.dense(predictions, 64, activation=tf.nn.tanh)
      predictions = tf.layers.dropout(predictions, rate=0.15)

      predictions = tf.layers.dense(predictions, 16, activation=tf.nn.tanh)
    
  else : 
      predictions = tf.layers.dense(input_layer, 128, activation=tf.nn.tanh)
      predictions = tf.layers.dense(predictions, 64, activation=tf.nn.tanh)
      predictions = tf.layers.dense(predictions, 16, activation=tf.nn.tanh)

  # continue  
  predictions = tf.layers.dense(predictions, 1, activation=None)  # X.W + b

  
  # 2. Loss function, training/eval ops
  if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
    
    labels = tf.expand_dims(tf.cast(labels, dtype=tf.float32), -1)
    
    loss = tf.losses.mean_squared_error(labels, predictions)
    
    optimizer = tf.train.FtrlOptimizer(learning_rate=0.2)
    
    train_op = optimizer.minimize(
      loss = loss,
      global_step = tf.train.get_global_step())
    
    eval_metric_ops = {
      "rmse": tf.metrics.root_mean_squared_error(labels, predictions)
    }
    
  else:
    loss = None
    train_op = None
    eval_metric_ops = None
  
  # 3. Create predictions
  predictions_dict = {"predicted": predictions}
  
  # 4. Create export outputs
  export_outputs = {"regression_export_outputs": tf.estimator.export.RegressionOutput(value = predictions)}
  
  # 5. Return EstimatorSpec
  return tf.estimator.EstimatorSpec(
      mode = mode,
      predictions = predictions_dict,
      loss = loss,
      train_op = train_op,
      eval_metric_ops = eval_metric_ops,
      export_outputs = export_outputs)

In [43]:
# Create serving input function
def serving_input_fn():
    
  feature_placeholders = {
      colname : tf.placeholder(tf.float32, [None]) for colname in 'housing_median_age,median_income,num_rooms,num_bedrooms,persons_per_house'.split(',')
  }
  feature_placeholders['longitude'] = tf.placeholder(tf.float32, [None])
  feature_placeholders['latitude'] = tf.placeholder(tf.float32, [None])
  
  features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
  }
    
  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [44]:
# Create custom estimator's train and evaluate function
def train_and_evaluate(output_dir):

  estimator = tf.estimator.Estimator(
    model_fn = custom_estimator, 
    model_dir = output_dir,
    params={'feature_columns': list(feature_columns.values())})
  
  train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn,
    max_steps = 10000)

  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  
  eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn,
    steps = None,
    exporters = exporter)
  
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)


In [45]:
#Run Training
OUTDIR = 'trained_model'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

train_and_evaluate(OUTDIR)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'trained_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5289d76e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Sta

INFO:tensorflow:global_step/sec: 282.397
INFO:tensorflow:loss = 0.33213812, step = 6801 (0.358 sec)
INFO:tensorflow:global_step/sec: 295.028
INFO:tensorflow:loss = 0.49506408, step = 6901 (0.336 sec)
INFO:tensorflow:global_step/sec: 277.701
INFO:tensorflow:loss = 0.3343081, step = 7001 (0.359 sec)
INFO:tensorflow:global_step/sec: 312.517
INFO:tensorflow:loss = 0.19991031, step = 7101 (0.322 sec)
INFO:tensorflow:global_step/sec: 266.624
INFO:tensorflow:loss = 0.23498175, step = 7201 (0.375 sec)
INFO:tensorflow:global_step/sec: 309.641
INFO:tensorflow:loss = 0.23826209, step = 7301 (0.324 sec)
INFO:tensorflow:global_step/sec: 270.564
INFO:tensorflow:loss = 0.30699256, step = 7401 (0.369 sec)
INFO:tensorflow:global_step/sec: 283.242
INFO:tensorflow:loss = 0.50278986, step = 7501 (0.353 sec)
INFO:tensorflow:global_step/sec: 297.497
INFO:tensorflow:loss = 0.41959423, step = 7601 (0.336 sec)
INFO:tensorflow:global_step/sec: 275.934
INFO:tensorflow:loss = 0.25563005, step = 7701 (0.365 sec)
I

In [46]:
# Launch tensorboard

# Note: If you use jupyter then you may have to use the address shown in the terminal console instead of the below link to tensorboard.

TensorBoard().start(OUTDIR)

TensorBoard was started successfully with pid 31821. Click here to access it.

31821